In [ ]:
import numpy as np
import pandas as pd 
import tensorflow as tf 
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler
import pickle

In [ ]:
#Load test data
data = pd.read_csv('data_output/processed_data.csv')
x = data.drop(columns=['label']).values # Features 
y = data['label'].values #Labels (1 = Normal, 0 = Abnormal)

#Load scaler
with open('models/scaler.pk1', 'rb') as f:
    scaler = pickle.load(f)

x_scaled = scaler.transform(x)

#Load trained autoencoder model
autoencoder = load_model('models/autoencoder.h5')

#Compute reconstruction errors
reconstructions = autoencoder.predict(x_scaled)
reconstruction_errors = np.mean(np.abs(reconstructions - x_scaled), axis=1)

#Compute threshold (95$ percentile of normal reconstruction errors)
def compute_threshold(autoencoder, x_train_normal):
    recontructions = autoencoder.predict(x_train_normal)
    reconstruction_errors = np.mean(np.abs(reconstructions - x_train_normal), axis=1):
    return np.percentile(reconstruction_errors, 95)

#Identify normal and abnormal data 
x_train_normal = x_scaled[y == 1]
threshold = compute_threshold(autoencoder, x_train_normal)
anomaly_prediction = reconstruction_errors > threshold

#Evaluate results 
num_anomalies = np.sum(anomaly_predictions)
print(f"Total anomalies detected: {num_anomalies}")

#Save predictions
data['Anomaly'] = anomaly_predictions.astype(int)
data.to_csv('data_output/evaluation_results.csv', index=False)
print("Evaluation results saved to data_output/evaluation_results.csv")